Previously On:

In [ ]:
import xarray as xr

path = '../../../data/'
file = path + 'thetao_Omon_historical_GISS-E2-1-G_r1i1p1f1_gn_185001-187012.nc'

ds = xr.open_dataset(file, chunks = {'time': 16})

da_thetao = ds['thetao']

We will look at this for only one point. Since ds.lev is a DataArray (ds alone is a DataSet), we can select data by in the index:

In [ ]:
level_point = ds.lev[0]
level_point

Let's one point in time using Xarray's `isel` method. `isel` refers to index-select and allows you to name the dimension in which you are subselecting. Read more about indexing and selecting data from an Xarray DataSet [here](http://xarray.pydata.org/en/stable/indexing.html).

In [ ]:
heat_time0 = heat.isel(time=0)
heat_time0

In [ ]:
heat_Jan_1860 = heat.sel(time=slice('1860-01-01','1860-02-01')).squeeze('time')
heat_Jan_1860

Now that we have our data, and have made sure the variables are in the correct units - let's use `xarray.DataArray.where` to filter our data to the depths where we want to integrate ocean heat content. This takes arguments as follows:

```python
da_filtered_2_conditions = da.where(condition, other = <NA>, drop = False)
```

Where the `other` and `drop` keyword specifies what you want to do with the data points that do not meet the condition.

So if you wanted to limit lev_bnds_m to the top 50 meters of the ocean depth you would type:

In [ ]:
level_bounds_limited = level_bounds_in_m.where(level_bounds_in_m < 50, drop = True)
level_bounds_limited.values

In [ ]:
## Task 5
Limit depth to the top 100 m in the code cell block below:

This is close, but we want that last bound to be 50, not 'nan'. So we will use the other keyword, instead of drop. This allows us to specify that we want all bounds that do not meet the condition, to be turned to the value 50.

In [ ]:
level_bounds_limited = level_bounds_in_m.where(level_bounds_in_m < 50, other = 50)
level_bounds_limited.values

But we don't want to carry around all these extra indices! We will now drop whenever the difference between the top and bottom of a level is 0.

To perform an integration, we need to find the step size (so the distance between each level bounds pair).

Python is zero-indexed so `a[:,0]` means values from all (`:`) rows in the zeroth (`0`) (or first or leftmost - however is easier for you to think of it) column. And `a[:,1]` means values from all rows in the next (rightmost) column.

In [ ]:
delta_level = abs(level_bounds_limited[:, 1] - level_bounds_limited[:, 0])
delta_level.values

Then we limit our delta_lev values to drop all values of zero.

In [ ]:
delta_level_limited = delta_level.where(delta_level != 0, drop = True)
delta_level_limited

## Task 6
We want to limit our temperature values by the same condition. Do so in the code cell block below:

Here is my solution:

In [ ]:
temperature_limited = temperature_in_degK.where(delta_level != 0, drop = True)

## Task 7
Let's turn this functionality into a function that repeats this work flow (replaces depth values above the limit with the limit value, finds the delta values for each level, and limits the level deltas and temperature coordinates to the depth limit) for any depth limit. 

Write your function in the code cell block below:

Mine looks like:

In [ ]:
def limit_depth_of_variables(level_bounds, temperature, depth_limit): 
    level_bounds_limited = level_bounds.where(level_bounds < depth_limit, depth_limit)
    delta_level = abs(level_bounds_limited[:, 1] - level_bounds_limited[:, 0])
    
    delta_level_limited = delta_level.where(delta_level != 0, drop = True)
    temperature_limited = temperature.where(delta_level != 0, drop = True)
    
    return delta_level_limited, temperature_limited

In [ ]:
delta_level_limited, temperature_limited = limit_depth_of_variables(level_bounds_in_m, temperature_in_degK, 50)
delta_level_limited, temperature_limited

<div class="alert alert-block alert-success">
  <p>Previous: <a href="01_modules_and_xarray_datasets.ipynb">Modules and Xarray Datasets</a></p>
  <p>Next: <a href="03_selecting_and_indexing_data.ipynb">Selecting and Indexing Data</a></p>
</div>